<a href="https://colab.research.google.com/github/Sarvveshp/NLP_assignment/blob/main/NLP_ass_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets rouge-score


In [3]:
from transformers import pipeline
from datasets import load_dataset
from rouge_score import rouge_scorer
import pandas as pd


In [4]:
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:1%]")
dataset = dataset.shuffle(seed=42).select(range(10))  # Use 10 samples for testing


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [5]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [6]:
def generate_summary(text):
    summary = summarizer(text, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
    return summary

# Example:
sample_article = dataset[0]['article']
print("Generated Summary:\n", generate_summary(sample_article))


Generated Summary:
 Superman, Batman, Wonder Woman, Captain America were created in the 1930s. A new exhibit in Los Angeles celebrates these icons from the Golden Age of Comic Books. A group of mostly Jewish artists created the costumed heroes who came to symbolize the hopes of a beleaguered nation.


In [7]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def evaluate():
    scores = []
    for item in dataset:
        ref = item['highlights']
        pred = generate_summary(item['article'])
        score = scorer.score(ref, pred)
        scores.append(score)
    return scores

results = evaluate()

# Calculate average ROUGE scores
avg_scores = {
    key: sum([r[key].fmeasure for r in results]) / len(results) for key in results[0]
}
print("Average ROUGE Scores:\n", avg_scores)


Average ROUGE Scores:
 {'rouge1': 0.38606236611773354, 'rouge2': 0.1681053690559869, 'rougeL': 0.31239444563165064}
